# Object Detection with Image or Webcam on Ryzen AI

This example demonstrates the object detection model inference on the embedded Neural Processing Unit (NPU) in your AMD Ryzen AI enabled PC with either single image or the live webcam. 

In [ ]:
# Before starting, be sure you've installed the requirements listed in the requirements.txt file:
!python -m pip install -r requirements.txt

### 1. Get Model from Ryzen AI model zoo
The yolov8 model from [Ryzen AI model zoo](https://huggingface.co/amd) will be applied in this example. You may choose any other object detection models with tiny difference in the pre and post processing.

In [ ]:
# Notebook dependencies
from huggingface_hub import hf_hub_download
from yolov8_utils import get_directories

current_dir = get_directories()

# Download Yolov8 model from Ryzen AI model zoo. Registration is required before download.
hf_hub_download(repo_id="amd/yolov8m", filename="yolov8m.onnx", local_dir=str(current_dir))


### 2. Model inference on CPU, iGPU and NPU with single image

In this session, we will leverage the onnxruntime to do the inference on CPU, iGPU and NPU with single image.
Here defines the pre and post processing functions which are the exactly the same on all OnnxRuntime EPs.

In [ ]:
import torch
import torch.nn as nn
import onnxruntime
import numpy as np
from yolov8_utils import *

def preprocess(img):
    img = torch.from_numpy(img)
    img = img.float()  # uint8 to fp16/32
    img /= 255  # 0 - 255 to 0.0 - 1.0
    return img


class DFL(nn.Module):
    # Integral module of Distribution Focal Loss (DFL) proposed in Generalized Focal Loss https://ieeexplore.ieee.org/document/9792391
    def __init__(self, c1=16):
        super().__init__()
        self.conv = nn.Conv2d(c1, 1, 1, bias=False).requires_grad_(False)
        x = torch.arange(c1, dtype=torch.float)
        self.conv.weight.data[:] = nn.Parameter(x.view(1, c1, 1, 1))
        self.c1 = c1

    def forward(self, x):
        b, c, a = x.shape  # batch, channels, anchors
        return self.conv(x.view(b, 4, self.c1, a).transpose(2, 1).softmax(1)).view(
            b, 4, a
        )


def dist2bbox(distance, anchor_points, xywh=True, dim=-1):
    """Transform distance(ltrb) to box(xywh or xyxy)."""
    lt, rb = torch.split(distance, 2, dim)
    x1y1 = anchor_points - lt
    x2y2 = anchor_points + rb
    if xywh:
        c_xy = (x1y1 + x2y2) / 2
        wh = x2y2 - x1y1
        return torch.cat((c_xy, wh), dim)  # xywh bbox
    return torch.cat((x1y1, x2y2), dim)  # xyxy bbox


def post_process(x):
    dfl = DFL(16)
    anchors = torch.tensor(
        np.load(
            "./anchors.npy",
            allow_pickle=True,
        )
    )
    strides = torch.tensor(
        np.load(
            "./strides.npy",
            allow_pickle=True,
        )
    )
    box, cls = torch.cat([xi.view(x[0].shape[0], 144, -1) for xi in x], 2).split(
        (16 * 4, 80), 1
    )
    dbox = dist2bbox(dfl(box), anchors.unsqueeze(0), xywh=True, dim=1) * strides
    y = torch.cat((dbox, cls.sigmoid()), 1)
    return y, x

# Load labels of coco dataaset
with open('coco.names', 'r') as f:
        names = f.read()

imgsz = [640, 640]

# Set input image
image_path = "sample_yolov8.jpg"
image = Image.open(image_path)

# Display the input image
image = image.resize((640,640), Image.BICUBIC)
display(image)

# Load image
dataset = LoadImages(
    image_path, imgsz=imgsz, stride=32, auto=False, transforms=None, vid_stride=1
)

#### CPU Inference

Inference with onnxruntime on CPU ep

In [ ]:
import onnxruntime
import numpy as np
import time
import matplotlib.pyplot as plt

# Specify the path to the quantized ONNX Model
onnx_model_path = "yolov8m.onnx"

# Output file
output_path = "cpu_result.jpg"

cpu_options = onnxruntime.SessionOptions()

# Create Inference Session to run the quantized model on the CPU
cpu_session = onnxruntime.InferenceSession(
    onnx_model_path,
    providers = ['CPUExecutionProvider'],
    sess_options=cpu_options,
)

for batch in dataset:
    path, im, im0s, vid_cap, s = batch
    im = preprocess(im)
    if len(im.shape) == 3:
        im = im[None]
    start = time.time()
    outputs = cpu_session.run(None, {cpu_session.get_inputs()[0].name: im.permute(0, 2, 3, 1).cpu().numpy()})
    end = time.time()

    inference_time = np.round((end - start) * 1000, 2)

    # Postprocessing
    outputs = [torch.tensor(item).permute(0, 3, 1, 2) for item in outputs]
    preds = post_process(outputs)
    preds = non_max_suppression(
        preds, 0.25, 0.7, agnostic=False, max_det=300, classes=None
    )
    plot_images(
        im,
        *output_to_target(preds, max_det=15),
        image_path,
        fname=output_path,
        names=names,
    )
    
    print('----------------------------------------')
    print('Inference time: ' + str(inference_time) + " ms")
    print('----------------------------------------')

#### iGPU Inference

We will leverage the onnxruntime DirectML ep to inference on AMD Radeon 780m iGPU

In [ ]:
# Specify the path to the quantized ONNX Model
onnx_model_path = "yolov8m.onnx"

# Output file
output_path = "dml_result.jpg"

dml_options = onnxruntime.SessionOptions()

# Create Inference Session to run the quantized model on the iGPU
dml_session = onnxruntime.InferenceSession(
    onnx_model_path,
    providers = ['DmlExecutionProvider'],
    provider_options = [{"device_id": "0"}]
)

for batch in dataset:
    path, im, im0s, vid_cap, s = batch
    im = preprocess(im)
    if len(im.shape) == 3:
        im = im[None]
    start = time.time()
    outputs = dml_session.run(None, {dml_session.get_inputs()[0].name: im.permute(0, 2, 3, 1).cpu().numpy()})
    end = time.time()

    inference_time = np.round((end - start) * 1000, 2)

    # Postprocessing
    outputs = [torch.tensor(item).permute(0, 3, 1, 2) for item in outputs]
    preds = post_process(outputs)
    preds = non_max_suppression(
        preds, 0.25, 0.7, agnostic=False, max_det=300, classes=None
    )
    plot_images(
        im,
        *output_to_target(preds, max_det=15),
        image_path,
        fname=output_path,
        names=names,
    )
    
    print('----------------------------------------')
    print('Inference time: ' + str(inference_time) + " ms")
    print('----------------------------------------')

#### NPU Inference

Inference with onnxruntime on NPU ep

In [ ]:
# Specify the path to the quantized ONNX Model
onnx_model_path = "yolov8m.onnx"

# Output file
output_path = "npu_result.jpg"

# Point to the config file path used for the VitisAI Execution Provider
config_file_path = "./vaip_config.json"

provider_options = [{
              'config_file': config_file_path,
              'ai_analyzer_visualization': True,
              'ai_analyzer_profiling': True,
          }]

npu_session = onnxruntime.InferenceSession(
    onnx_model_path,
    providers = ['VitisAIExecutionProvider'],
    provider_options = provider_options
)

for batch in dataset:
    path, im, im0s, vid_cap, s = batch
    im = preprocess(im)
    if len(im.shape) == 3:
        im = im[None]
    start = time.time()
    outputs = npu_session.run(None, {npu_session.get_inputs()[0].name: im.permute(0, 2, 3, 1).cpu().numpy()})
    end = time.time()

    inference_time = np.round((end - start) * 1000, 2)

    # Postprocessing
    outputs = [torch.tensor(item).permute(0, 3, 1, 2) for item in outputs]
    preds = post_process(outputs)
    preds = non_max_suppression(
        preds, 0.25, 0.7, agnostic=False, max_det=300, classes=None
    )
    plot_images(
        im,
        *output_to_target(preds, max_det=15),
        image_path,
        fname=output_path,
        names=names,
    )
    
    print('----------------------------------------')
    print('Inference time: ' + str(inference_time) + " ms")
    print('----------------------------------------')

### 3. Model inference on NPU with webcam

Now we have validated the the model with image., and we will use the webcam as live input to do the inference on NPU.

In [ ]:
import torch
import torch.nn as nn
import onnxruntime
import numpy as np
import cv2
from yolov8_utils import *

# display videos in notebook
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output, display, HTML


def frame_process(frame, input_shape=(640, 640)):
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, input_shape)
    img = torch.from_numpy(img)
    img = img.float()  # uint8 to fp16/32
    img /= 255  # 0 - 255 to 0.0 - 1.0
    img = np.transpose(img, (2, 0, 1))
    return img
    

class DFL(nn.Module):
    # Integral module of Distribution Focal Loss (DFL) proposed in Generalized Focal Loss https://ieeexplore.ieee.org/document/9792391
    def __init__(self, c1=16):
        super().__init__()
        self.conv = nn.Conv2d(c1, 1, 1, bias=False).requires_grad_(False)
        x = torch.arange(c1, dtype=torch.float)
        self.conv.weight.data[:] = nn.Parameter(x.view(1, c1, 1, 1))
        self.c1 = c1

    def forward(self, x):
        b, c, a = x.shape  # batch, channels, anchors
        return self.conv(x.view(b, 4, self.c1, a).transpose(2, 1).softmax(1)).view(
            b, 4, a
        )


def dist2bbox(distance, anchor_points, xywh=True, dim=-1):
    """Transform distance(ltrb) to box(xywh or xyxy)."""
    lt, rb = torch.split(distance, 2, dim)
    x1y1 = anchor_points - lt
    x2y2 = anchor_points + rb
    if xywh:
        c_xy = (x1y1 + x2y2) / 2
        wh = x2y2 - x1y1
        return torch.cat((c_xy, wh), dim)  # xywh bbox
    return torch.cat((x1y1, x2y2), dim)  # xyxy bbox


def post_process(x):
    dfl = DFL(16)
    anchors = torch.tensor(
        np.load(
            "./anchors.npy",
            allow_pickle=True,
        )
    )
    strides = torch.tensor(
        np.load(
            "./strides.npy",
            allow_pickle=True,
        )
    )
    box, cls = torch.cat([xi.view(x[0].shape[0], 144, -1) for xi in x], 2).split(
        (16 * 4, 80), 1
    )
    dbox = dist2bbox(dfl(box), anchors.unsqueeze(0), xywh=True, dim=1) * strides
    y = torch.cat((dbox, cls.sigmoid()), 1)
    return y, x

# Load labels of coco dataaset
with open('coco.names', 'r') as f:
        names = f.read()

imgsz = [640, 640]

# Specify the path to the quantized ONNZ Model
onnx_model_path = "yolov8m.onnx"

# Point to the config file path used for the VitisAI Execution Provider
config_file_path = "./vaip_config.json"

npu_options = onnxruntime.SessionOptions()

npu_session = onnxruntime.InferenceSession(
    onnx_model_path,
    providers = ['VitisAIExecutionProvider'],
    sess_options=npu_options,
    provider_options=[{'config_file': config_file_path}]
)

with open('coco.names', 'r') as f:
        names = f.read()

# Video input
cap = cv2.VideoCapture(0)

while (True):
    try:
        clear_output(wait=True)
        ret, frame = cap.read()
        
        if not ret:
            break
            
        input_shape = (640, 640)

        im = frame_process(frame, input_shape)
        if len(im.shape) == 3:
            im = im[None]
        outputs = npu_session.run(None, {npu_session.get_inputs()[0].name: im.permute(0, 2, 3, 1).cpu().numpy()})

        # Postprocessing
        outputs = [torch.tensor(item).permute(0, 3, 1, 2) for item in outputs]
        preds = post_process(outputs)
        preds = non_max_suppression(
            preds, 0.25, 0.7, agnostic=False, max_det=300, classes=None
        )

        colors = [[random.randint(0, 255) for _ in range(3)] 
                for _ in range(len(names))]

        plot_images(
        im,
        *output_to_target(preds, max_det=15),
        frame,
        fname="output.jpg",
        names=names,
        )
        
    except KeyboardInterrupt:
        cap.release()

4. Model Analysis on NPU
After NPU inference, there are several '.json' files generated by the Ryzen AI tracing tool, which could be open by the AI Analyzer for further optimization.

In [ ]:
!aianalyzer ./ -p 8001